In [ ]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib as plt

Read the training data

In [ ]:
df = pd.read_csv("./data/train.csv") #Reading the dataset in a dataframe using Pandas
df.describe() # get the summary of numerical variables

Sum the nulls fields in every column

In [ ]:
 df.apply(lambda x: sum(x.isnull()),axis=0) 

Fill the empty(nulls) Self_Employed column cells

In [ ]:
df['Self_Employed'].fillna('No',inplace=True)

In [ ]:
table = df.pivot_table(values='LoanAmount', index='Self_Employed' ,columns='Education', aggfunc=np.median)
table

In [ ]:
def fage(x):
 return table.loc[x['Self_Employed'],x['Education']]

Fill the empty(null cells) LoanAmount column

In [ ]:
df['LoanAmount'].fillna(df[df['LoanAmount'].isnull()].apply(fage, axis=1), inplace=True)

In [ ]:
df['Loan_Amount_Term'].fillna(360, inplace=True)
df['Credit_History'].fillna(1, inplace=True)
df['Dependents'].fillna(0, inplace=True)
df['Married'].fillna('Yes', inplace=True) 
df['Gender'].fillna('Male', inplace=True)

In [ ]:
df.dtypes

Converting categories to numeric

In [ ]:
from sklearn.preprocessing import LabelEncoder
var_mod = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i].astype(str))

In [ ]:
df.dtypes

Generic classification function

In [ ]:
#Import models from scikit learn module:
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
  #Fit the model:
  model.fit(data[predictors],data[outcome])
  
  #Make predictions on training set:
  predictions = model.predict(data[predictors])
  
  #Print accuracy
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print("Accuracy : %s" % "{0:.3%}".format(accuracy))

  #Perform k-fold cross-validation with 5 folds
  kf = KFold(data.shape[0], n_folds=5)
  error = []
  for train, test in kf:
    # Filter training data
    train_predictors = (data[predictors].iloc[train,:])
    
    # The target we're using to train the algorithm.
    train_target = data[outcome].iloc[train]
    
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)
    
    #Record error from each cross-validation run
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
  print("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))

  #Fit the model again so that it can be refered outside the function:
  model.fit(data[predictors],data[outcome])

##GaussianNB Algorithm:
We choosed 'GaussianNB' Algorithm and train the model
Naive Bayes is a simple technique for constructing classifiers: models that assign class labels to problem instances, represented as vectors of feature values, where the class labels are drawn from some finite set.
When dealing with continuous data, a typical assumption is that the continuous values associated with each class are distributed according to a Gaussian distribution

In [ ]:
outcome_var = 'Loan_Status'
model = GaussianNB()
predictor_var = ['Credit_History', 'LoanAmount', 'Property_Area']
classification_model(model, df,predictor_var,outcome_var)

Read the test data

In [ ]:
df_test = pd.read_csv("./data/test.csv")
df_loan_ids = df_test['Loan_ID']
df_test

Fill null cells and convert the categories to numeric

In [ ]:
df_test['LoanAmount'].fillna(df_test[df_test['LoanAmount'].isnull()].apply(fage, axis=1), inplace=True)
df_test['Loan_Amount_Term'].fillna(360, inplace=True)
df_test['Credit_History'].fillna(1, inplace=True)
df_test['Gender'].fillna('Male', inplace=True) 
df_test['Married'].fillna('Yes', inplace=True) 
df_test['Dependents'].fillna(0, inplace=True)
var_mod = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']
le = LabelEncoder()
for i in var_mod:
    df_test[i] = le.fit_transform(df_test[i].astype(str))

Take the relvant categories

In [ ]:
df_test = df_test[predictor_var]
df_test

Predict the test

In [ ]:
result = model.predict(df_test)
result

Convert the result to the desired output

In [ ]:
def convert(x):
    if x == 1:
        return 'Y'
    else:
        return 'N'

In [ ]:
result = [convert(res) for res in result]

Write the output file

In [ ]:

df_to_write = {'Loan_ID' : df_loan_ids , 'Loan_Status' : result}
df_to_write = pd.DataFrame(df_to_write)
df_to_write.to_csv("Sample_Submission1.csv")